In [1]:
import requests
existing_url = 'https://www.imdb.com/chart/top/'
response = requests.get(existing_url)
print(response.status_code)

200


In [3]:
response = requests.get(existing_url)
print(response.content.decode('utf-8'))




<!DOCTYPE html>
<html
    xmlns:og="http://ogp.me/ns#"
    xmlns:fb="http://www.facebook.com/2008/fbml">
    <head>
         

        <meta charset="utf-8">
        <meta http-equiv="X-UA-Compatible" content="IE=edge">

    
    
    

    
    
    




        <script type="text/javascript">var IMDbTimer={starttime: new Date().getTime(),pt:'java'};</script>

<script>
    if (typeof uet == 'function') {
      uet("bb", "LoadTitle", {wb: 1});
    }
</script>
  <script>(function(t){ (t.events = t.events || {})["csm_head_pre_title"] = new Date().getTime(); })(IMDbTimer);</script>
        <title>Top 250 Movies - IMDb</title>
  <script>(function(t){ (t.events = t.events || {})["csm_head_post_title"] = new Date().getTime(); })(IMDbTimer);</script>
<script>
    if (typeof uet == 'function') {
      uet("be", "LoadTitle", {wb: 1});
    }
</script>
<script>
    if (typeof uex == 'function') {
      uex("ld", "LoadTitle", {wb: 1});
    }
</script>

        <link rel="canonical" href="https:

In [4]:
from bs4 import BeautifulSoup
document = response.content
soup = BeautifulSoup(document, 'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<html xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://ogp.me/ns#">
 <head>
  <meta charset="utf-8"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <script type="text/javascript">
   var IMDbTimer={starttime: new Date().getTime(),pt:'java'};
  </script>
  <script>
   if (typeof uet == 'function') {
      uet("bb", "LoadTitle", {wb: 1});
    }
  </script>
  <script>
   (function(t){ (t.events = t.events || {})["csm_head_pre_title"] = new Date().getTime(); })(IMDbTimer);
  </script>
  <title>
   Top 250 Movies - IMDb
  </title>
  <script>
   (function(t){ (t.events = t.events || {})["csm_head_post_title"] = new Date().getTime(); })(IMDbTimer);
  </script>
  <script>
   if (typeof uet == 'function') {
      uet("be", "LoadTitle", {wb: 1});
    }
  </script>
  <script>
   if (typeof uex == 'function') {
      uex("ld", "LoadTitle", {wb: 1});
    }
  </script>
  <link href="https://www.imdb.com/chart/top/" rel="canonical"/>
  <meta content="http://w

In [5]:
soup.find_all('a')

[<a href="/?ref_=nv_home"><svg class="ipc-logo drawer-logo" height="56" version="1.1" viewbox="0 0 64 32" width="98" xmlns="http://www.w3.org/2000/svg"><g fill="#F5C518"><rect height="100%" rx="4" width="100%" x="0" y="0"></rect></g><g fill="#000000" fill-rule="nonzero" transform="translate(8.000000, 7.000000)"><polygon points="0 18 5 18 5 0 0 0"></polygon><path d="M15.6725178,0 L14.5534833,8.40846934 L13.8582008,3.83502426 C13.65661,2.37009263 13.4632474,1.09175121 13.278113,0 L7,0 L7,18 L11.2416347,18 L11.2580911,6.11380679 L13.0436094,18 L16.0633571,18 L17.7583653,5.8517865 L17.7707076,18 L22,18 L22,0 L15.6725178,0 Z"></path><path d="M24,18 L24,0 L31.8045586,0 C33.5693522,0 35,1.41994415 35,3.17660424 L35,14.8233958 C35,16.5777858 33.5716617,18 31.8045586,18 L24,18 Z M29.8322479,3.2395236 C29.6339219,3.13233348 29.2545158,3.08072342 28.7026524,3.08072342 L28.7026524,14.8914865 C29.4312846,14.8914865 29.8796736,14.7604764 30.0478195,14.4865461 C30.2159654,14.2165858 30.3021941,13.486

In [22]:
titlelist=[]
for url in soup.find_all('a'):
        titlelist.append(url.get('href'))
titlelist2=[]
substring = "title/tt"
for item in titlelist:
    try:
        item.index(substring)
    except ValueError:
        pass
    except AttributeError:
        pass
    else:
        titlelist2.append(item)
titlelist_url = []
for i in titlelist2:
    if i not in titlelist_url:
        titlelist_url.append(i)
titlelist_url

['/title/tt0111161/',
 '/title/tt0068646/',
 '/title/tt0468569/',
 '/title/tt0071562/',
 '/title/tt0050083/',
 '/title/tt0108052/',
 '/title/tt0167260/',
 '/title/tt0110912/',
 '/title/tt0120737/',
 '/title/tt0060196/',
 '/title/tt0109830/',
 '/title/tt0137523/',
 '/title/tt1375666/',
 '/title/tt0167261/',
 '/title/tt0080684/',
 '/title/tt0133093/',
 '/title/tt0099685/',
 '/title/tt0073486/',
 '/title/tt0114369/',
 '/title/tt0047478/',
 '/title/tt0038650/',
 '/title/tt0102926/',
 '/title/tt0120815/',
 '/title/tt0317248/',
 '/title/tt0118799/',
 '/title/tt0120689/',
 '/title/tt0076759/',
 '/title/tt0816692/',
 '/title/tt0103064/',
 '/title/tt0088763/',
 '/title/tt0245429/',
 '/title/tt0054215/',
 '/title/tt0110413/',
 '/title/tt0253474/',
 '/title/tt6751668/',
 '/title/tt0110357/',
 '/title/tt0172495/',
 '/title/tt0120586/',
 '/title/tt0114814/',
 '/title/tt0407887/',
 '/title/tt0482571/',
 '/title/tt0034583/',
 '/title/tt2582802/',
 '/title/tt1675434/',
 '/title/tt0027977/',
 '/title/t

In [7]:
titles=soup.find_all('td', class_="titleColumn")
titles

[<td class="titleColumn">
       1.
       <a href="/title/tt0111161/" title="Frank Darabont (dir.), Tim Robbins, Morgan Freeman">A remény rabjai</a>
 <span class="secondaryInfo">(1994)</span>
 </td>,
 <td class="titleColumn">
       2.
       <a href="/title/tt0068646/" title="Francis Ford Coppola (dir.), Marlon Brando, Al Pacino">A keresztapa</a>
 <span class="secondaryInfo">(1972)</span>
 </td>,
 <td class="titleColumn">
       3.
       <a href="/title/tt0468569/" title="Christopher Nolan (dir.), Christian Bale, Heath Ledger">A sötét lovag</a>
 <span class="secondaryInfo">(2008)</span>
 </td>,
 <td class="titleColumn">
       4.
       <a href="/title/tt0071562/" title="Francis Ford Coppola (dir.), Al Pacino, Robert De Niro">A keresztapa II</a>
 <span class="secondaryInfo">(1974)</span>
 </td>,
 <td class="titleColumn">
       5.
       <a href="/title/tt0050083/" title="Sidney Lumet (dir.), Henry Fonda, Lee J. Cobb">Tizenkét dühös ember</a>
 <span class="secondaryInfo">(1957)</spa

In [8]:
top20_titles=[]
for i in range(20):
    temptitle=titles[i].find_all('a')[0]
    start = str(temptitle).find(">") + len(">")
    end = str(temptitle).find("</a>")
    titlename = str(temptitle)[start:end]
    top20_titles.append(titlename)
top20_titles

['A remény rabjai',
 'A keresztapa',
 'A sötét lovag',
 'A keresztapa II',
 'Tizenkét dühös ember',
 'Schindler listája',
 'A Gyűrűk Ura: A király visszatér',
 'Ponyvaregény',
 'A Gyűrűk Ura: A gyűrű szövetsége',
 'A Jó, a Rossz és a Csúf',
 'Forrest Gump',
 'Harcosok klubja',
 'Eredet',
 'A Gyűrűk Ura: A két torony',
 'A Birodalom visszavág',
 'Mátrix',
 'Nagymenők',
 'Száll a kakukk fészkére',
 'Hetedik',
 'A hét szamuráj']

In [9]:
ratingslist=[]
ratingslist=soup.find_all('td', class_="ratingColumn")
ratingslist

[<td class="ratingColumn imdbRating">
 <strong title="9.2 based on 2,567,174 user ratings">9.2</strong>
 </td>,
 <td class="ratingColumn">
 <div class="seen-widget seen-widget-tt0111161 pending" data-titleid="tt0111161">
 <div class="boundary">
 <div class="popover">
 <span class="delete"> </span><ol><li>1<li>2<li>3<li>4<li>5<li>6<li>7<li>8<li>9<li>10</li></li></li></li></li></li></li></li></li></li></ol> </div>
 </div>
 <div class="inline">
 <div class="pending"></div>
 <div class="unseeable">NOT YET RELEASED</div>
 <div class="unseen"> </div>
 <div class="rating"></div>
 <div class="seen">Seen</div>
 </div>
 </div>
 </td>,
 <td class="ratingColumn imdbRating">
 <strong title="9.2 based on 1,767,186 user ratings">9.2</strong>
 </td>,
 <td class="ratingColumn">
 <div class="seen-widget seen-widget-tt0068646 pending" data-titleid="tt0068646">
 <div class="boundary">
 <div class="popover">
 <span class="delete"> </span><ol><li>1<li>2<li>3<li>4<li>5<li>6<li>7<li>8<li>9<li>10</li></li></li

In [10]:
top20_ratings=[]
for i in range(20):
    temptitle=ratingslist[2*i].find_all('strong')[0]
    start = str(temptitle).find('title="') + len('title="')
    end = str(temptitle).find(" based")
    rankname = str(temptitle)[start:end]
    top20_ratings.append(float(rankname))
top20_ratings

[9.2,
 9.2,
 9.0,
 9.0,
 9.0,
 8.9,
 8.9,
 8.9,
 8.8,
 8.8,
 8.8,
 8.8,
 8.7,
 8.7,
 8.7,
 8.7,
 8.7,
 8.6,
 8.6,
 8.6]

In [11]:
top20_ratingnum=[]
for i in range(20):
    temptitle=ratingslist[2*i].find_all('strong')[0]
    start = str(temptitle).find('on ') + len('on ')
    end = str(temptitle).find(" user")
    rankname = str(temptitle)[start:end]
    rankname = rankname.replace(",","")
    top20_ratingnum.append(int(rankname))
top20_ratingnum

[2567174,
 1767186,
 2531214,
 1222804,
 758183,
 1308160,
 1766314,
 1970781,
 1788052,
 738501,
 1980116,
 2019976,
 2253086,
 1595747,
 1242200,
 1847918,
 1107827,
 978155,
 1574027,
 337318]

In [23]:
start_url = 'https://www.imdb.com'
end_url = titlelist_url[0]
url=start_url+end_url
url

'https://www.imdb.com/title/tt0111161/'

In [24]:
response2=requests.get(url)
document2 = response2.content
soupurl = BeautifulSoup(document2, 'html.parser')
print(soupurl.prettify())

<!DOCTYPE html>
<html lang="en-US" xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://opengraphprotocol.org/schema/">
 <head>
  <meta content="width=device-width" name="viewport"/>
  <meta charset="utf-8"/>
  <title>
   A remény rabjai (1994) - IMDb
  </title>
  <meta content="A remény rabjai: Directed by Frank Darabont. With Tim Robbins, Morgan Freeman, Bob Gunton, William Sadler. Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency." data-id="main" name="description"/>
  <meta content="Reviews, Showtimes, DVDs, Photos, User Ratings, Synopsis, Trailers, Credits" name="keywords"/>
  <script type="application/ld+json">
   {"@context":"https://schema.org","@type":"Movie","url":"/title/tt0111161/","name":"The Shawshank Redemption","alternateName":"A remény rabjai","image":"https://m.media-amazon.com/images/M/MV5BMDFkYTc0MGEtZmNhMC00ZDIzLWFmNTEtODM1ZmRlYWMwMWFmXkEyXkFqcGdeQXVyMTMxODk2OTU@._V1_.jpg","description":"T

In [35]:
oscarsoup=list(soupurl.find_all('a', class_="ipc-metadata-list-item__label ipc-metadata-list-item__label--link"))
oscarsoup

[<a class="ipc-metadata-list-item__label ipc-metadata-list-item__label--link" href="/title/tt0111161/fullcredits/cast?ref_=tt_ov_st_sm" rel="" target="">Stars</a>,
 <a class="ipc-metadata-list-item__label ipc-metadata-list-item__label--link" href="/title/tt0111161/fullcredits/cast?ref_=tt_ov_st_sm" rel="" target="">Stars</a>,
 <a class="ipc-metadata-list-item__label ipc-metadata-list-item__label--link" href="/title/tt0111161/awards/?ref_=tt_awd" rel="" target="">Nominated for 7 Oscars</a>,
 <a class="ipc-metadata-list-item__label ipc-metadata-list-item__label--link" href="/title/tt0111161/fullcredits/?ref_=tt_cl_sm" rel="" target="">All cast &amp; crew</a>,
 <a class="ipc-metadata-list-item__label ipc-metadata-list-item__label--link" href="https://pro.imdb.com/title/tt0111161/?rf=cons_tt_btf_cc&amp;ref_=cons_tt_btf_cc" rel="" target="_blank">See more cast details at IMDbPro</a>,
 <a class="ipc-metadata-list-item__label ipc-metadata-list-item__label--link" href="/title/tt0111161/parenta

In [85]:
start_url = 'https://www.imdb.com'
oscarlist=[]
substring_oscar="Oscar"
for end_url in titlelist_url[0:20]:
    url=start_url+end_url
    response2=requests.get(url)
    document2 = response2.content
    soupurl = BeautifulSoup(document2, 'html.parser')
    oscarsouplist=list(soupurl.find_all('a', class_="ipc-metadata-list-item__label ipc-metadata-list-item__label--link"))
    tempchecker=0
    for item2 in oscarsouplist:
        if substring_oscar in str(item2):
            tempchecker=1
    if tempchecker == 0:
        oscarlist.append("No Oscar")
    else:
        for item in oscarsouplist:
            try:
                str(item).index(substring_oscar)
            except ValueError:
                pass
            except AttributeError:
                pass
            else:
                oscarlist.append(str(item))
oscarlist

['<a class="ipc-metadata-list-item__label ipc-metadata-list-item__label--link" href="/title/tt0111161/awards/?ref_=tt_awd" rel="" target="">Nominated for 7 Oscars</a>',
 '<a class="ipc-metadata-list-item__label ipc-metadata-list-item__label--link" href="/title/tt0068646/awards/?ref_=tt_awd" rel="" target="">Won 3 Oscars</a>',
 '<a class="ipc-metadata-list-item__label ipc-metadata-list-item__label--link" href="/title/tt0468569/awards/?ref_=tt_awd" rel="" target="">Won 2 Oscars</a>',
 '<a class="ipc-metadata-list-item__label ipc-metadata-list-item__label--link" href="/title/tt0071562/awards/?ref_=tt_awd" rel="" target="">Won 6 Oscars</a>',
 '<a class="ipc-metadata-list-item__label ipc-metadata-list-item__label--link" href="/title/tt0050083/awards/?ref_=tt_awd" rel="" target="">Nominated for 3 Oscars</a>',
 '<a class="ipc-metadata-list-item__label ipc-metadata-list-item__label--link" href="/title/tt0108052/awards/?ref_=tt_awd" rel="" target="">Won 7 Oscars</a>',
 '<a class="ipc-metadata-l

In [86]:
len(oscarlist)

20

In [87]:
number_of_oscars = []
won_oscars = "Won"
for item in oscarlist:
    if won_oscars in item:
        start = str(item).find("Won ") + len("Won ")
        end = str(item).find(" Oscar")
        oscar_num = str(item)[start:end]
        number_of_oscars.append(int(oscar_num))
    else:
        number_of_oscars.append(0)
number_of_oscars

[0, 3, 2, 6, 0, 7, 11, 1, 4, 0, 6, 0, 4, 2, 1, 4, 1, 5, 0, 0]

In [88]:
len(number_of_oscars)

20

In [89]:
ranks=[]
for i in range(20):
    ranks.append(i+1)
ranks

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

In [91]:
import pandas as pd
df = pd.DataFrame(list(zip(ranks, top20_titles,top20_ratings,top20_ratingnum,number_of_oscars)),
               columns =['Original Rank', 'Title','Original Rating','Number of Ratings','Oscars Won'])
df

,Original Rank,Title,Original Rating,Number of Ratings,Oscars Won
0,1,A remény rabjai,9.2,2567174,0
1,2,A keresztapa,9.2,1767186,3
2,3,A sötét lovag,9.0,2531214,2
3,4,A keresztapa II,9.0,1222804,6
4,5,Tizenkét dühös ember,9.0,758183,0
5,6,Schindler listája,8.9,1308160,7
6,7,A Gyűrűk Ura: A király visszatér,8.9,1766314,11
7,8,Ponyvaregény,8.9,1970781,1
8,9,A Gyűrűk Ura: A gyűrű szövetsége,8.8,1788052,4
9,10,"A Jó, a Rossz és a Csúf",8.8,738501,0
